## Turkish Instructions Retrieval Embedding

In [1]:
# !pip install datasets

### Import Libraries

In [3]:
import sys
import os

import pandas as pd
import numpy as np
from utils import missing_values
import warnings
from datetime import datetime

from datasets import load_dataset

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

warnings.filterwarnings('ignore')

#### Set Path and Constant Values

In [2]:
# Set Constants

# If use Google Colab
use_colab = True
if use_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    ROOT_PATH = "/content/drive/MyDrive/KollektifOgrenme/Hw3-Retrieval/BONUS"  # Google Drive için kök dizin
    sys.path.append(os.path.join(ROOT_PATH, 'src'))
    device = "cuda"
else:
    ROOT_PATH = os.getcwd()
    device = "cpu"

CACHE_PATH = "/content/drive/MyDrive/Colab Notebooks/HW3/cache"
DATA_PATH = os.path.join(ROOT_PATH, 'data')
RAW_DATA_PATH = os.path.join(DATA_PATH, 'raw')
EMBEDDING_DATA_PATH = os.path.join(DATA_PATH, 'embedding')

# Create Folders
os.makedirs(RAW_DATA_PATH, exist_ok=True)
os.makedirs(EMBEDDING_DATA_PATH, exist_ok=True)
os.makedirs(CACHE_PATH, exist_ok=True)

Mounted at /content/drive


### Load Dataset: Databricks Dolly 15K TR

In [5]:
# https://huggingface.co/datasets/atasoglu/databricks-dolly-15k-tr
# atasoglu/databricks-dolly-15k-tr

ds = load_dataset("atasoglu/databricks-dolly-15k-tr")

README.md:   0%|          | 0.00/341 [00:00<?, ?B/s]

databricks-dolly-15k-tr.jsonl:   0%|          | 0.00/14.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15014 [00:00<?, ? examples/s]

In [6]:
df_ins_data_all = ds['train'].to_pandas()
df_ins_data_all.head()

,instruction,context,response,category
0,Virgin Australia ne zaman faaliyete geçti?,"Virgin Australia Airlines Pty Ltd'nin ticari adı olan Virgin Australia, Avustralya merkezli bir havayolu şirketidir. Filo büyüklüğüne göre Virgin markasını kullanan en büyük havayolu şirketidir. 31 Ağustos 2000'de Virgin Blue adıyla, tek hat üzerinde iki uçakla hizmete başlamıştır.[3] Eylül 2001'de Ansett Australia'nın iflas etmesinden sonra birdenbire kendisini Avustralya iç pazarında önemli bir havayolu şirketi olarak buldu. Havayolu o zamandan beri Brisbane, Melbourne ve Sidney'deki merkezlerden Avustralya'nın 32 şehrine doğrudan hizmet verecek şekilde büyüdü.[4]","Virgin Australia, 31 Ağustos 2000'de Virgin Blue olarak, tek bir rotada iki uçakla hizmete başladı.",closed_qa
1,Hangisi bir balık türüdür? Tope veya İp,None,top,classification
2,Develer neden susuz uzun süre yaşayabilir?,None,"Develer, hörgüçlerini uzun süre enerji ve su ile doldurmak için hörgüçlerindeki yağı kullanırlar.",open_qa
3,"Alice'in ailesinin üç kızı var: Amy, Jessy ve üçüncü kızın adı nedir?",None,Üçüncü kızının adı Alice'dir.,open_qa
4,Tomoaki Komorida ne zaman doğdu?,"Komorida, 10 Temmuz 1981'de Kumamoto Eyaletinde doğdu. Liseden mezun olduktan sonra 2000 yılında J1 League kulübü Avispa Fukuoka'ya katıldı. 2001'de orta saha oyuncusu olarak çıkış yapmasına rağmen fazla oynamadı ve kulüp 1. 2001 sezonunun sonunda J2 Ligi. 2002'de J2 kulübü Oita Trinita'ya taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu oldu ve kulüp 2002'de şampiyonluğu kazandı ve 2003'te terfi etti. 2005'e kadar birçok maç oynadı. Eylül 2005'te J2 kulübü Montedio Yamagata'ya geçti. 2006'da J2 kulübü Vissel Kobe'ye taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu olmasına rağmen, yaz aylarında giderek daha az oynandı. 2007'de, yerel bölgesinde bulunan Japonya Futbol Ligi kulübü Rosso Kumamoto'ya (daha sonra Roasso Kumamoto) taşındı. Normal bir oyuncu olarak oynadı ve kulüp 2008'de J2'ye yükseldi. Eskisi kadar oynamasa da yine de birçok maçta forma giydi. 2010 yılında Endonezya'ya taşındı ve Persela Lamongan'a katıldı. Temmuz 2010'da Japonya'ya döndü ve J2 kulübü Giravanz Kitakyushu'ya katıldı. Emekli olduğu 2012 yılına kadar sık ​​sık defansif orta saha oyuncusu ve stoper olarak oynadı.","Tomoaki Komorida, 10 Temmuz 1981'de doğdu.",closed_qa


In [7]:
df_ins_data_all["category"].value_counts()

,count
category,
open_qa,3611
general_qa,2191
classification,2136
closed_qa,1823
brainstorming,1767
information_extraction,1512
summarization,1263
creative_writing,711


In [8]:
df_ins_data_all.columns

Index(['instruction', 'context', 'response', 'category'], dtype='object')

In [9]:
df_ins_data_all.rename(columns={"instruction":"talimat",
                                "context":"giris",
                                "response":"cevap"}, inplace=True)
# df_ins_data_all.drop(labels=["category"])

In [10]:
ds_name = "databricks-dolly-15k-tr"
os.makedirs(os.path.join(RAW_DATA_PATH, ds_name), exist_ok=True)

ds_file_name = os.path.join(RAW_DATA_PATH, ds_name, "turkish_instructions.csv")
df_ins_data_all.to_csv(ds_file_name, index=False, encoding='utf-8')
df_ins_data_all.head()

,talimat,giris,cevap,category
0,Virgin Australia ne zaman faaliyete geçti?,"Virgin Australia Airlines Pty Ltd'nin ticari adı olan Virgin Australia, Avustralya merkezli bir havayolu şirketidir. Filo büyüklüğüne göre Virgin markasını kullanan en büyük havayolu şirketidir. 31 Ağustos 2000'de Virgin Blue adıyla, tek hat üzerinde iki uçakla hizmete başlamıştır.[3] Eylül 2001'de Ansett Australia'nın iflas etmesinden sonra birdenbire kendisini Avustralya iç pazarında önemli bir havayolu şirketi olarak buldu. Havayolu o zamandan beri Brisbane, Melbourne ve Sidney'deki merkezlerden Avustralya'nın 32 şehrine doğrudan hizmet verecek şekilde büyüdü.[4]","Virgin Australia, 31 Ağustos 2000'de Virgin Blue olarak, tek bir rotada iki uçakla hizmete başladı.",closed_qa
1,Hangisi bir balık türüdür? Tope veya İp,None,top,classification
2,Develer neden susuz uzun süre yaşayabilir?,None,"Develer, hörgüçlerini uzun süre enerji ve su ile doldurmak için hörgüçlerindeki yağı kullanırlar.",open_qa
3,"Alice'in ailesinin üç kızı var: Amy, Jessy ve üçüncü kızın adı nedir?",None,Üçüncü kızının adı Alice'dir.,open_qa
4,Tomoaki Komorida ne zaman doğdu?,"Komorida, 10 Temmuz 1981'de Kumamoto Eyaletinde doğdu. Liseden mezun olduktan sonra 2000 yılında J1 League kulübü Avispa Fukuoka'ya katıldı. 2001'de orta saha oyuncusu olarak çıkış yapmasına rağmen fazla oynamadı ve kulüp 1. 2001 sezonunun sonunda J2 Ligi. 2002'de J2 kulübü Oita Trinita'ya taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu oldu ve kulüp 2002'de şampiyonluğu kazandı ve 2003'te terfi etti. 2005'e kadar birçok maç oynadı. Eylül 2005'te J2 kulübü Montedio Yamagata'ya geçti. 2006'da J2 kulübü Vissel Kobe'ye taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu olmasına rağmen, yaz aylarında giderek daha az oynandı. 2007'de, yerel bölgesinde bulunan Japonya Futbol Ligi kulübü Rosso Kumamoto'ya (daha sonra Roasso Kumamoto) taşındı. Normal bir oyuncu olarak oynadı ve kulüp 2008'de J2'ye yükseldi. Eskisi kadar oynamasa da yine de birçok maçta forma giydi. 2010 yılında Endonezya'ya taşındı ve Persela Lamongan'a katıldı. Temmuz 2010'da Japonya'ya döndü ve J2 kulübü Giravanz Kitakyushu'ya katıldı. Emekli olduğu 2012 yılına kadar sık ​​sık defansif orta saha oyuncusu ve stoper olarak oynadı.","Tomoaki Komorida, 10 Temmuz 1981'de doğdu.",closed_qa


### Show Metadata, Shape, Statistics and Missing Table

In [11]:
df_ins_data_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15014 entries, 0 to 15013
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   talimat   15014 non-null  object
 1   giris     4598 non-null   object
 2   cevap     15014 non-null  object
 3   category  15014 non-null  object
dtypes: object(4)
memory usage: 469.3+ KB


In [12]:
df_ins_data_all.shape

(15014, 4)

In [13]:
missing_values(df=df_ins_data_all, threshold=0, asc_sorting=False)

,Count,Percent
Columns,,
giris,10416,69.37525
talimat,0,0.00000
cevap,0,0.00000
category,0,0.00000


In [14]:
col_list = df_ins_data_all.columns.to_list()

for col in col_list:
    df_ins_data_all[f"{col}Length"] = df_ins_data_all[col].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)
    df_ins_data_all[f"{col}WordCount"] = df_ins_data_all[col].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)

In [15]:
df_ins_data_all.head()

,talimat,giris,cevap,category,talimatLength,talimatWordCount,girisLength,girisWordCount,cevapLength,cevapWordCount,categoryLength,categoryWordCount
0,Virgin Australia ne zaman faaliyete geçti?,"Virgin Australia Airlines Pty Ltd'nin ticari adı olan Virgin Australia, Avustralya merkezli bir havayolu şirketidir. Filo büyüklüğüne göre Virgin markasını kullanan en büyük havayolu şirketidir. 31 Ağustos 2000'de Virgin Blue adıyla, tek hat üzerinde iki uçakla hizmete başlamıştır.[3] Eylül 2001'de Ansett Australia'nın iflas etmesinden sonra birdenbire kendisini Avustralya iç pazarında önemli bir havayolu şirketi olarak buldu. Havayolu o zamandan beri Brisbane, Melbourne ve Sidney'deki merkezlerden Avustralya'nın 32 şehrine doğrudan hizmet verecek şekilde büyüdü.[4]","Virgin Australia, 31 Ağustos 2000'de Virgin Blue olarak, tek bir rotada iki uçakla hizmete başladı.",closed_qa,42,6,572,73,99,15,9,1
1,Hangisi bir balık türüdür? Tope veya İp,None,top,classification,39,7,0,0,3,1,14,1
2,Develer neden susuz uzun süre yaşayabilir?,None,"Develer, hörgüçlerini uzun süre enerji ve su ile doldurmak için hörgüçlerindeki yağı kullanırlar.",open_qa,42,6,0,0,97,13,7,1
3,"Alice'in ailesinin üç kızı var: Amy, Jessy ve üçüncü kızın adı nedir?",None,Üçüncü kızının adı Alice'dir.,open_qa,69,12,0,0,29,4,7,1
4,Tomoaki Komorida ne zaman doğdu?,"Komorida, 10 Temmuz 1981'de Kumamoto Eyaletinde doğdu. Liseden mezun olduktan sonra 2000 yılında J1 League kulübü Avispa Fukuoka'ya katıldı. 2001'de orta saha oyuncusu olarak çıkış yapmasına rağmen fazla oynamadı ve kulüp 1. 2001 sezonunun sonunda J2 Ligi. 2002'de J2 kulübü Oita Trinita'ya taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu oldu ve kulüp 2002'de şampiyonluğu kazandı ve 2003'te terfi etti. 2005'e kadar birçok maç oynadı. Eylül 2005'te J2 kulübü Montedio Yamagata'ya geçti. 2006'da J2 kulübü Vissel Kobe'ye taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu olmasına rağmen, yaz aylarında giderek daha az oynandı. 2007'de, yerel bölgesinde bulunan Japonya Futbol Ligi kulübü Rosso Kumamoto'ya (daha sonra Roasso Kumamoto) taşındı. Normal bir oyuncu olarak oynadı ve kulüp 2008'de J2'ye yükseldi. Eskisi kadar oynamasa da yine de birçok maçta forma giydi. 2010 yılında Endonezya'ya taşındı ve Persela Lamongan'a katıldı. Temmuz 2010'da Japonya'ya döndü ve J2 kulübü Giravanz Kitakyushu'ya katıldı. Emekli olduğu 2012 yılına kadar sık ​​sık defansif orta saha oyuncusu ve stoper olarak oynadı.","Tomoaki Komorida, 10 Temmuz 1981'de doğdu.",closed_qa,32,5,1131,163,42,6,9,1


In [16]:
df_ins_data_all[["talimatLength", "girisLength", "cevapLength", "talimatWordCount", "girisWordCount", "cevapWordCount"]].describe(percentiles=[0.80, 0.85, 0.90, 0.95, 0.99]).T

,count,mean,std,min,50%,80%,85%,90%,95%,99%,max
talimatLength,15014.0,73.018316,136.167348,4.0,54.0,96.0,111.0,131.0,169.00,286.00,11914.0
girisLength,15014.0,354.883842,956.829778,0.0,0.0,601.0,796.0,1092.0,1698.70,3975.70,23864.0
cevapLength,15014.0,358.318836,594.308380,1.0,185.0,512.0,622.0,807.0,1198.70,2644.44,26097.0
talimatWordCount,15014.0,9.841415,14.361268,1.0,8.0,13.0,15.0,18.0,22.00,38.00,870.0
girisWordCount,15014.0,46.654389,130.877607,0.0,0.0,79.0,105.0,143.0,220.35,507.61,3796.0
cevapWordCount,15014.0,48.086319,80.956700,1.0,25.0,68.0,83.0,107.0,158.35,351.00,4274.0


In [17]:
# df_ins_data_all[df_ins_data_all["talimatWordCount"]==1]

In [18]:
# df_ins_data_all[df_ins_data_all["talimatWordCount"]==870]

### Split Data

In [19]:
df_ins_data_all['soru'] = df_ins_data_all['talimat'].fillna('') + ' ' + df_ins_data_all['giris'].fillna('')
df_ins_data_all[["talimat", "giris", "soru", "cevap"]].head()

,talimat,giris,soru,cevap
0,Virgin Australia ne zaman faaliyete geçti?,"Virgin Australia Airlines Pty Ltd'nin ticari adı olan Virgin Australia, Avustralya merkezli bir havayolu şirketidir. Filo büyüklüğüne göre Virgin markasını kullanan en büyük havayolu şirketidir. 31 Ağustos 2000'de Virgin Blue adıyla, tek hat üzerinde iki uçakla hizmete başlamıştır.[3] Eylül 2001'de Ansett Australia'nın iflas etmesinden sonra birdenbire kendisini Avustralya iç pazarında önemli bir havayolu şirketi olarak buldu. Havayolu o zamandan beri Brisbane, Melbourne ve Sidney'deki merkezlerden Avustralya'nın 32 şehrine doğrudan hizmet verecek şekilde büyüdü.[4]","Virgin Australia ne zaman faaliyete geçti? Virgin Australia Airlines Pty Ltd'nin ticari adı olan Virgin Australia, Avustralya merkezli bir havayolu şirketidir. Filo büyüklüğüne göre Virgin markasını kullanan en büyük havayolu şirketidir. 31 Ağustos 2000'de Virgin Blue adıyla, tek hat üzerinde iki uçakla hizmete başlamıştır.[3] Eylül 2001'de Ansett Australia'nın iflas etmesinden sonra birdenbire kendisini Avustralya iç pazarında önemli bir havayolu şirketi olarak buldu. Havayolu o zamandan beri Brisbane, Melbourne ve Sidney'deki merkezlerden Avustralya'nın 32 şehrine doğrudan hizmet verecek şekilde büyüdü.[4]","Virgin Australia, 31 Ağustos 2000'de Virgin Blue olarak, tek bir rotada iki uçakla hizmete başladı."
1,Hangisi bir balık türüdür? Tope veya İp,None,Hangisi bir balık türüdür? Tope veya İp,top
2,Develer neden susuz uzun süre yaşayabilir?,None,Develer neden susuz uzun süre yaşayabilir?,"Develer, hörgüçlerini uzun süre enerji ve su ile doldurmak için hörgüçlerindeki yağı kullanırlar."
3,"Alice'in ailesinin üç kızı var: Amy, Jessy ve üçüncü kızın adı nedir?",None,"Alice'in ailesinin üç kızı var: Amy, Jessy ve üçüncü kızın adı nedir?",Üçüncü kızının adı Alice'dir.
4,Tomoaki Komorida ne zaman doğdu?,"Komorida, 10 Temmuz 1981'de Kumamoto Eyaletinde doğdu. Liseden mezun olduktan sonra 2000 yılında J1 League kulübü Avispa Fukuoka'ya katıldı. 2001'de orta saha oyuncusu olarak çıkış yapmasına rağmen fazla oynamadı ve kulüp 1. 2001 sezonunun sonunda J2 Ligi. 2002'de J2 kulübü Oita Trinita'ya taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu oldu ve kulüp 2002'de şampiyonluğu kazandı ve 2003'te terfi etti. 2005'e kadar birçok maç oynadı. Eylül 2005'te J2 kulübü Montedio Yamagata'ya geçti. 2006'da J2 kulübü Vissel Kobe'ye taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu olmasına rağmen, yaz aylarında giderek daha az oynandı. 2007'de, yerel bölgesinde bulunan Japonya Futbol Ligi kulübü Rosso Kumamoto'ya (daha sonra Roasso Kumamoto) taşındı. Normal bir oyuncu olarak oynadı ve kulüp 2008'de J2'ye yükseldi. Eskisi kadar oynamasa da yine de birçok maçta forma giydi. 2010 yılında Endonezya'ya taşındı ve Persela Lamongan'a katıldı. Temmuz 2010'da Japonya'ya döndü ve J2 kulübü Giravanz Kitakyushu'ya katıldı. Emekli olduğu 2012 yılına kadar sık ​​sık defansif orta saha oyuncusu ve stoper olarak oynadı.","Tomoaki Komorida ne zaman doğdu? Komorida, 10 Temmuz 1981'de Kumamoto Eyaletinde doğdu. Liseden mezun olduktan sonra 2000 yılında J1 League kulübü Avispa Fukuoka'ya katıldı. 2001'de orta saha oyuncusu olarak çıkış yapmasına rağmen fazla oynamadı ve kulüp 1. 2001 sezonunun sonunda J2 Ligi. 2002'de J2 kulübü Oita Trinita'ya taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu oldu ve kulüp 2002'de şampiyonluğu kazandı ve 2003'te terfi etti. 2005'e kadar birçok maç oynadı. Eylül 2005'te J2 kulübü Montedio Yamagata'ya geçti. 2006'da J2 kulübü Vissel Kobe'ye taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu olmasına rağmen, yaz aylarında giderek daha az oynandı. 2007'de, yerel bölgesinde bulunan Japonya Futbol Ligi kulübü Rosso Kumamoto'ya (daha sonra Roasso Kumamoto) taşındı. Normal bir oyuncu olarak oynadı ve kulüp 2008'de J2'ye yükseldi. Eskisi kadar oynamasa da yine de birçok maçta forma giydi. 2010 yılında Endonezya'ya taşındı ve Persela Lamongan'a katıldı. Temmuz

In [20]:
df_sample_1 = df_ins_data_all[df_ins_data_all['giris'].notna()].sample(n=1000, random_state=42)
df_sample_1_idx = df_sample_1.index
df_sample_1.head()

df_sample_2 = df_ins_data_all[~(df_ins_data_all['giris'].notna())].sample(n=1000, random_state=42)
df_sample_2_idx = df_sample_2.index
df_sample_2.head()

df_sample = pd.concat([df_sample_1, df_sample_2])
df_sample['soru'] = df_sample['talimat'].fillna('') + " " + df_sample['giris'].fillna('')

df_ins_data_all[col].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)

sampling_file_name = os.path.join(RAW_DATA_PATH, ds_name, "sampled_turkish_instructions.csv")
df_sample.to_csv(sampling_file_name, index=False)

df_sample = df_sample[["talimat", "giris", "soru", "cevap"]].reset_index(drop=True)

df_sample = pd.read_csv(sampling_file_name)
df_sample.head()

talimat  \
0                                                                                           Kara ceviz ağaçları nerede yetişir?   
1                                                                     Gunshy albümünü yayınladığında Josh Logan kaç yaşındaydı?   
2  Aşağıdaki paragrafa göre, TENS (transkutanöz elektriksel sinir stimülasyonu) tedavisinin bazı potansiyel faydaları nelerdir?   
3                                                                                                            Baur au Lac nedir?   
4                                                                                                Nero ne zaman ve nerede doğdu?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [21]:
df_sample.sample(5)

,talimat,giris,cevap,category,talimatLength,talimatWordCount,girisLength,girisWordCount,cevapLength,cevapWordCount,categoryLength,categoryWordCount,soru
1909,Wyoming hangi yılda eyalet oldu?,NaN,1890,open_qa,32,5,0,0,4,1,7,1,Wyoming hangi yılda eyalet oldu?
7,Alice Twemlow nerede konuk eleştirmen oldu?,"Alice Twemlow, çalışmaları grafik tasarıma odaklanan Birleşik Krallık'tan bir yazar, eleştirmen ve eğitimcidir. Yale University School of Art, Maryland Institute College of Art (MICA) ve Rhode Island School of Design'da (RISD) konuk eleştirmen olarak yer aldı.[1] 2006 yılında, New York'taki Görsel Sanatlar Okulu (SVA), Twemlow'u Tasarım Eleştirisi Alanında Güzel Sanatlar Yüksek Lisans Programı'nın (D-Crit) başkanı ve kurucu ortağı olarak atadı.[2] SVA biyografisine göre: ""Alice Twemlow, Eye, Design Issues, I.D., Print, New York dergisi ve The Architect's Newspaper için yazıyor."" [3] Twemlow ayrıca Voice: AIGA Journal of Design adlı çevrimiçi yayına da katkıda bulunmaktadır.[4] 2012'de Core77, onu Core77 Tasarım Ödülleri'nin “Tasarım Yazma ve Yorumlama” kategorisi için jüri kaptanı olarak seçti.[5] Twemlow, 2017-2018'de Design Academy Eindhoven'da Tasarım Küratörlüğü ve Yazarlığı alanında MA başkanıydı ve şu anda Lahey'deki Kraliyet Güzel Sanatlar Akademisi'nde (KABK) Lector Design[6] ve Leiden Üniversitesi'nde Doçent.[ 7][8]","Yale Üniversitesi Sanat Okulu'nda, Maryland Enstitüsü Sanat Okulu'nda ve Rhode Island Tasarım Okulu'nda.",information_extraction,43,6,1040,137,104,13,22,1,"Alice Twemlow nerede konuk eleştirmen oldu? Alice Twemlow, çalışmaları grafik tasarıma odaklanan Birleşik Krallık'tan bir yazar, eleştirmen ve eğitimcidir. Yale University School of Art, Maryland Institute College of Art (MICA) ve Rhode Island School of Design'da (RISD) konuk eleştirmen olarak yer aldı.[1] 2006 yılında, New York'taki Görsel Sanatlar Okulu (SVA), Twemlow'u Tasarım Eleştirisi Alanında Güzel Sanatlar Yüksek Lisans Programı'nın (D-Crit) başkanı ve kurucu ortağı olarak atadı.[2] SVA biyografisine göre: ""Alice Twemlow, Eye, Design Issues, I.D., Print, New York dergisi ve The Architect's Newspaper için yazıyor."" [3] Twemlow ayrıca Voice: AIGA Journal of Design adlı çevrimiçi yayına da katkıda bulunmaktadır.[4] 2012'de Core77, onu Core77 Tasarım Ödülleri'nin “Tasarım Yazma ve Yorumlama” kategorisi için jüri kaptanı olarak seçti.[5] Twemlow, 2017-2018'de Design Academy Eindhoven'da Tasarım Küratörlüğü ve Yazarlığı alanında MA başkanıydı ve şu anda Lahey'deki Kraliyet Güzel Sanatlar Akademisi'nde (KABK) Lector Design[6] ve Leiden Üniversitesi'nde Doçent.[ 7][8]"
584,"Referans metne göre, Howard Frankland köprüsündeki yeniden yapılanma projesinin başlangıç ​​tarihi ve beklenen tamamlanma tarihi neydi?","1992'deki genişletme projesinden sonra Interstate 275, köprünün kendisinde sekiz şeride çıkarıldı. Ancak bu, köprünün her iki ucundaki kapasiteyi artırmadı. Yedeklemeler, esas olarak SR 60/Veterans Expressway çıkışındaki bir darboğaz nedeniyle Tampa'ya giden Howard Frankland'da hala görülüyordu. St.Petersburg tarafında, on yıldan fazla süren kapsamlı bir yeniden inşa projesinin ardından, şerit sayısı köprünün önündeki dört şeritten St. Petersburg şehir merkezinden geçen altı şeride ve Gandy Bulvarı'ndan köprüye giden sekiz şeride çıkarıldı.\n\nPinellas County'deki farklı bölümleri birbirine bağlamak için yeni bir paralı yol inşa etme planı olan yeni Gateway Expressway projesi için 2017 yılında bir yeniden inşa projesinin başlaması planlandı. Ancak FDOT, eyaletler arası otoyolu orijinal daha büyük iki fazlı proje yerine daha küçük aşamalarda yeniden inşa etmeyi planladı ve inşaatın başlaması 2020'ye ertelendi.\n\nYeniden yapılanma projesi 2025'te bittiğinde, Howard Frankland köprüsündeki büyük trafik sıkışıklığının yeni şeritlerin eklenmesiyle önemli ölçüde azalması bekleniyor.[8] 7 Ocak 2021'de FDOT, bir üst geçidin ve I-275 üzerinden ilgili çıkış rampasının kaldırılması için başlangıç ​​tarihini bir hafta 16 Ocak 2021'e erteledi. 9 Ocak EST i

In [22]:
df_sample.shape

(2000, 13)

### Embedding Models

#### Description

<!-- ![Alt text]("Embedding") -->

| **No** | **Model Adı**                           | **Parametre Sayısı** | **Boyut Sayısı (Dimension Size)** | **Açıklama (Description)**                                                                                     | **Özel Amaçlı Eğitim**                   | **Eğitim Veri Kümesi**                                                        | **Desteklenen Diller**        |
|--------|----------------------------------------|----------------------|----------------------------------|---------------------------------------------------------------------------------------------------------------|-----------------------------------------|-----------------------------------------------------------------------------|---------------------------------|
| 1      | **sentence-transformers/all-MiniLM-L12-v2** | 33.4M                | 384                              | Cümleleri ve kısa paragrafları 384 boyutlu yoğun bir vektör uzayına dönüştüren bir modeldir. Bu model, cümle benzerliği, kümeleme ve anlamsal arama gibi görevlerde kullanılabilir. ([HuggingFace](https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2)) | Evet, microsoft/MiniLM-L12-H384-uncased modeli üzerinde ince ayar yapılmıştır. | Cümle çiftleri (NLI ve Paraphrase Corpus)                                     | İngilizce                       |
| 2      | **jinaai/jina-embeddings-v3**           | 572M                 | 1536                             | Jina AI tarafından geliştirilen bu model, metinleri 1536 boyutlu vektörlere dönüştürmek için tasarlanmıştır. ([HuggingFace](https://huggingface.co/jinaai/jina-embeddings-v3))         | -                                       | Özel büyük ölçekli çok dilli veri kümesi                                      | Çok Dilli (50+ dil)            |
| 3      | **intfloat/multilingual-e5-large-instruct** | 560M                 | 1024                             | Çok dilli metinleri 1024 boyutlu vektörlere dönüştürebilen bir modeldir. ([HuggingFace](https://huggingface.co/intfloat/multilingual-e5-large-instruct))                                | -                                       | mC4, CCNet, MassiveText gibi büyük veri kümeleri                              | Çok Dilli (100+ dil)           |
| 4      | **BAAI/bge-m3**                         | -                    | 1024                             | Beijing Academy of Artificial Intelligence (BAAI) tarafından geliştirilen bu model, metinleri 1024 boyutlu vektörlere dönüştürmek için tasarlanmıştır. ([HuggingFace](https://huggingface.co/BAAI/bge-m3)) | -                                       | OpenAI GPT benzeri veri kümesi (web crawl)                                    | İngilizce, Çince               |
| 5      | **ytu-ce-cosmos/turkish-colbert**       | 111M                 | 128                              | Yıldız Teknik Üniversitesi tarafından geliştirilen bu model, Türkçe metinleri 128 boyutlu vektörlere dönüştürmek için tasarlanmıştır. ([HuggingFace](https://huggingface.co/ytu-ce-cosmos/turkish-colbert)) | Evet, Türkçe metinler üzerinde özel olarak eğitilmiştir. | Türkçe metin veri kümeleri                                                  | Türkçe                         |
| 6      | **thenlper/gte-large**                  | 335M                 | 768                              | The NLP ER tarafından geliştirilen bu model, metinleri 768 boyutlu vektörlere dönüştürmek için tasarlanmıştır. ([HuggingFace](https://huggingface.co/thenlper/gte-large))             | -                                       | Çoklu cümle çiftleri ve NLI veri kümeleri                                     | İngilizce                       |


In [23]:
from sentence_transformers import SentenceTransformer

embedding_models = [
    'sentence-transformers/all-MiniLM-L12-v2',
    "jinaai/jina-embeddings-v3",
    "intfloat/multilingual-e5-large-instruct",
    "BAAI/bge-m3",
    "ytu-ce-cosmos/turkish-colbert",
    # "thenlper/gte-large"
]

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [11]:
def load_or_download_model(model_name, device):
    model_path = os.path.join(CACHE_PATH, model_name)
    if os.path.exists(model_path):
        print(f"Loading Cached Model: {model_name}")
        return SentenceTransformer(model_path, device=device, trust_remote_code=True)
    else:
        print(f"Downloading Model: {model_name}")
        model = SentenceTransformer(model_name, device=device, trust_remote_code=True)
        model.save(model_path)
        return model

In [31]:
ds_name = "databricks-dolly-15k-tr"
os.makedirs(os.path.join(EMBEDDING_DATA_PATH, ds_name), exist_ok=True)

In [26]:
# df_sample = df_sample[0:5]

In [27]:
for idx, model_name in enumerate(embedding_models):
    try:
        embedding_start_time = datetime.now()
        print(f"{idx+1}-Embedding Model Name: {model_name} - {embedding_start_time}")
        model = load_or_download_model(model_name, device)

        # Sorular için embedding
        print("Sorular için embedding hesaplanıyor...")
        # if embedding_models == "ytu-ce-cosmos/turkish-colbert":
        #     df_sample['soru'] = df_sample['soru'].str.lower()
        question_embeddings = model.encode(df_sample['soru'].tolist(), show_progress_bar=True)
        question_embedding_dim = len(question_embeddings[0])
        print(f"Embedding Dimension: {question_embedding_dim}")
        question_columns = [f'question_embedding_{i}' for i in range(question_embedding_dim)]
        question_df = pd.DataFrame(question_embeddings, columns=question_columns)
        question_df = pd.concat([df_sample[['soru']], question_df], axis=1)

        question_output_file = os.path.join(
          EMBEDDING_DATA_PATH, ds_name, f"question_{model_name.replace('/', '__')}_embeddings.csv.gz"
        )
        question_df.to_csv(question_output_file, compression='gzip', index=False)

        # Cevaplar için embedding
        print("Cevaplar için embedding hesaplanıyor...")
        # if embedding_models == "ytu-ce-cosmos/turkish-colbert":
        #     df_sample['soru'] = df_sample['soru'].str.lower()
        answer_embeddings = model.encode(df_sample['cevap'].tolist(), show_progress_bar=True)
        answer_embedding_dim = len(answer_embeddings[0])
        print(f"Embedding Dimension: {answer_embedding_dim}")
        answer_columns = [f'answer_embedding_{i}' for i in range(answer_embedding_dim)]
        answer_df = pd.DataFrame(answer_embeddings, columns=answer_columns)
        answer_df = pd.concat([df_sample[['cevap']], answer_df], axis=1)
        answer_output_file = os.path.join(
          EMBEDDING_DATA_PATH, ds_name, f"answer_{model_name.replace('/', '__')}_embeddings.csv.gz"
        )
        answer_df.to_csv(answer_output_file, compression='gzip', index=False)

        print(f"{model_name} için embedding işlemi tamamlandı. Embedding Duration: {datetime.now() - embedding_start_time}")
        print("+ " * 15)
    except Exception as exc:
        print(f"{model_name} için hata oluştu: {exc}")
        print("x " * 15)

1-Embedding Model Name: sentence-transformers/all-MiniLM-L12-v2 - 2024-12-22 06:54:55.983770
Loading Cached Model: sentence-transformers/all-MiniLM-L12-v2
Sorular için embedding hesaplanıyor...


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Embedding Dimension: 384
Cevaplar için embedding hesaplanıyor...


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Embedding Dimension: 384
sentence-transformers/all-MiniLM-L12-v2 için embedding işlemi tamamlandı. Embedding Duration: 0:00:33.136786
+ + + + + + + + + + + + + + + 
2-Embedding Model Name: jinaai/jina-embeddings-v3 - 2024-12-22 06:55:29.121499
Loading Cached Model: jinaai/jina-embeddings-v3


...

Embedding Dimension: 768
ytu-ce-cosmos/turkish-colbert için embedding işlemi tamamlandı. Embedding Duration: 0:00:36.962148
+ + + + + + + + + + + + + + + 
6-Embedding Model Name: thenlper/gte-large - 2024-12-22 07:01:23.851180
Loading Cached Model: thenlper/gte-large
Sorular için embedding hesaplanıyor...


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Embedding Dimension: 1024
Cevaplar için embedding hesaplanıyor...


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Embedding Dimension: 1024
thenlper/gte-large için embedding işlemi tamamlandı. Embedding Duration: 0:01:21.576950
+ + + + + + + + + + + + + + + 


In [24]:
from sentence_transformers import SentenceTransformer

embedding_models = [
    # 'sentence-transformers/all-MiniLM-L12-v2',
    "jinaai/jina-embeddings-v3",
    # "intfloat/multilingual-e5-large-instruct",
    # "BAAI/bge-m3",
    # "ytu-ce-cosmos/turkish-colbert",
    #"thenlper/gte-large"
]

In [36]:
for idx, model_name in enumerate(embedding_models):
    try:
        embedding_start_time = datetime.now()
        print(f"{idx+1}-Embedding Model Name: {model_name} - {embedding_start_time}")
        model = load_or_download_model(model_name, device)

        # Sorular için embedding
        print("Sorular için embedding hesaplanıyor...")
        # if embedding_models == "ytu-ce-cosmos/turkish-colbert":
        #     df_sample['soru'] = df_sample['soru'].str.lower()
        question_embeddings = model.encode(df_sample['soru'].tolist(), show_progress_bar=True, batch_size = 5)
        question_embedding_dim = len(question_embeddings[0])
        print(f"Embedding Dimension: {question_embedding_dim}")
        question_columns = [f'question_embedding_{i}' for i in range(question_embedding_dim)]
        question_df = pd.DataFrame(question_embeddings, columns=question_columns)
        question_df = pd.concat([df_sample[['soru']], question_df], axis=1)

        question_output_file = os.path.join(
          EMBEDDING_DATA_PATH, ds_name, f"question_{model_name.replace('/', '__')}_embeddings.csv.gz"
        )
        question_df.to_csv(question_output_file, compression='gzip', index=False)

        # Cevaplar için embedding
        print("Cevaplar için embedding hesaplanıyor...")
        # if embedding_models == "ytu-ce-cosmos/turkish-colbert":
        #     df_sample['soru'] = df_sample['soru'].str.lower()
        answer_embeddings = model.encode(df_sample['cevap'].tolist(), show_progress_bar=True, batch_size = 5)
        answer_embedding_dim = len(answer_embeddings[0])
        print(f"Embedding Dimension: {answer_embedding_dim}")
        answer_columns = [f'answer_embedding_{i}' for i in range(answer_embedding_dim)]
        answer_df = pd.DataFrame(answer_embeddings, columns=answer_columns)
        answer_df = pd.concat([df_sample[['cevap']], answer_df], axis=1)
        answer_output_file = os.path.join(
          EMBEDDING_DATA_PATH, ds_name, f"answer_{model_name.replace('/', '__')}_embeddings.csv.gz"
        )
        answer_df.to_csv(answer_output_file, compression='gzip', index=False)

        print(f"{model_name} için embedding işlemi tamamlandı. Embedding Duration: {datetime.now() - embedding_start_time}")
        print("+ " * 15)
    except Exception as exc:
        print(f"{model_name} için hata oluştu: {exc}")
        print("x " * 15)

1-Embedding Model Name: jinaai/jina-embeddings-v3 - 2024-12-22 07:30:01.585859
Loading Cached Model: jinaai/jina-embeddings-v3


Sorular için embedding hesaplanıyor...


Batches:   0%|          | 0/400 [00:00<?, ?it/s]

Embedding Dimension: 1024
Cevaplar için embedding hesaplanıyor...


Batches:   0%|          | 0/400 [00:00<?, ?it/s]

Embedding Dimension: 1024
jinaai/jina-embeddings-v3 için embedding işlemi tamamlandı. Embedding Duration: 0:01:06.369795
+ + + + + + + + + + + + + + + 


#### Embedding Vectors

In [ ]:
# df_emb_data_que_1 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "question_sentence-transformers__all-MiniLM-L12-v2_embeddings.csv.gz"), compression='gzip')
# df_emb_data_ans_1 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "answer_sentence-transformers__all-MiniLM-L12-v2_embeddings.csv.gz"), compression='gzip')

# df_emb_data_que_2 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "question_jinaai__jina-embeddings-v3_embeddings.csv.gz"), compression='gzip')
# df_emb_data_ans_2 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "answer_jinaai__jina-embeddings-v3_embeddings.csv.gz"), compression='gzip')

# df_emb_data_que_3 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "question_intfloat__multilingual-e5-large-instruct_embeddings.csv.gz"), compression='gzip')
# df_emb_data_ans_3 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "answer_intfloat__multilingual-e5-large-instruct_embeddings.csv.gz"), compression='gzip')

# df_emb_data_que_4 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "question_BAAI__bge-m3_embeddings.csv.gz"), compression='gzip')
# df_emb_data_ans_4 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "answer_BAAI__bge-m3_embeddings.csv.gz"), compression='gzip')

# df_emb_data_que_5 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "question_ytu-ce-cosmos__turkish-colbert_embeddings.csv.gz"), compression='gzip')
# df_emb_data_ans_5 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "answer_ytu-ce-cosmos__turkish-colbert_embeddings.csv.gz"), compression='gzip')

# df_emb_data_que_6 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "question_thenlper__gte-large_embeddings.csv.gz"), compression='gzip')
# df_emb_data_ans_6 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "answer_thenlper__gte-large_embeddings.csv.gz"), compression='gzip')

In [45]:
for idx, f in enumerate(os.listdir(EMBEDDING_DATA_PATH + "/" + ds_name)):
    # print(f)
    # if f.split("_")[0] == "answer":
    if f.split("_")[0] in ["answer", "question"]:
        df_emb_data = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH + "/" + ds_name, f), compression='gzip')
        print(f"{idx+1} - Emb. File Name:{f} & Shape: {df_emb_data.shape}")
        print(df_emb_data.head(0))
        # print(f"Columns:{df_emb_data.columns.to_list()}")
        print("+ " * 25)

1 - Emb. File Name:question_sentence-transformers__all-MiniLM-L12-v2_embeddings.csv.gz & Shape: (2000, 385)
Empty DataFrame
Columns: [soru, question_embedding_0, question_embedding_1, question_embedding_2, question_embedding_3, question_embedding_4, question_embedding_5, question_embedding_6, question_embedding_7, question_embedding_8, question_embedding_9, question_embedding_10, question_embedding_11, question_embedding_12, question_embedding_13, question_embedding_14, question_embedding_15, question_embedding_16, question_embedding_17, question_embedding_18, question_embedding_19, question_embedding_20, question_embedding_21, question_embedding_22, question_embedding_23, question_embedding_24, question_embedding_25, question_embedding_26, question_embedding_27, question_embedding_28, question_embedding_29, question_embedding_30, question_embedding_31, question_embedding_32, question_embedding_33, question_embedding_34, question_embedding_35, question_embedding_36, question_embedding_

In [46]:
EMBEDDING_DATA_PATH + "/" + ds_name

'/content/drive/MyDrive/KollektifOgrenme/Hw3-Retrieval/BONUS/data/embedding/databricks-dolly-15k-tr'